In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm_notebook
from sys import argv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime

In [3]:
test_path = 'poker-hand-testing.data'
train_path = 'poker-hand-training-true.data'

In [10]:
def read_data(path):
    data = pd.read_csv(path, header=None)
    return data

In [14]:
train_data = read_data(train_path)
train_data

,0,1,2,3,4,5,6,7,8,9,10
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9
3,4,10,4,11,4,1,4,13,4,12,9
4,4,1,4,13,4,12,4,11,4,10,9
...,...,...,...,...,...,...,...,...,...,...,...
25005,3,9,2,6,4,11,4,12,2,4,0
25006,4,1,4,10,3,13,3,4,1,10,1
25007,2,1,2,10,4,4,4,1,4,13,1
25008,2,12,4,3,1,10,1,12,4,9,1


In [15]:
test_data = read_data(test_path)
test_data

,0,1,2,3,4,5,6,7,8,9,10
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0
...,...,...,...,...,...,...,...,...,...,...,...
999995,3,1,1,12,2,9,4,9,2,6,1
999996,3,3,4,5,2,7,1,4,4,3,1
999997,1,11,4,7,3,9,1,13,2,7,1
999998,3,11,1,8,1,1,3,13,2,8,1


In [22]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(train_data.iloc[:,:-1])

OneHotEncoder(handle_unknown='ignore')

In [29]:
X_train = ohe.transform(train_data.iloc[:,:-1])
Y_train = np.array(train_data.iloc[:,-1])
print(X_train.shape, Y_train.shape)
print('unique y:',np.unique(Y_train))

(25010, 85) (25010,)
unique y: [0 1 2 3 4 5 6 7 8 9]


In [30]:
X_test = ohe.transform(test_data.iloc[:,:-1])
Y_test = np.array(test_data.iloc[:,-1])
print(X_test.shape, Y_test.shape)
print('uniq y:', np.unique(Y_test))

(1000000, 85) (1000000,)
uniq y: [0 1 2 3 4 5 6 7 8 9]


In [ ]:
class NN:
    def __init__(self, layers, feature_size = 85, target_class = 10, lr = 0.1, activatoin_m = 'sigmoid'):
        self.hidden_layers = layers
        self.total_layers  = len(layers) + 1
        self.feature_size  = feature_size
        self.target_class  = target_class
        self.activation_m  = activation_m
        self.lr = lr
        self.params = []
    
    def activation(self, X, typ = 'sigmoid'):
        if typ == 'sigmoid':
            return 1.0/(1.0 + np.exp(-X))
        else:
            return np.where(X<0,0,X)
        
    def softmax(self,X):
        exps = np.exp(X)
        return exps / np.sum(exps, axis=1).reshape(-1,1)
    
    def grad_act(self, X, typ = 'sigmoid'):
        if typ == 'sigmoid':
            return X*(1-X)
        else:
            return 1 * (X > 0)
    
    def initialize_params(self):
        total_layers = self.layers.append(self.target_class)
        layer_ip = self.feature_size
        for layer_no, layer_op in enumerate(total_layers):
            local_w = np.random.randn(layer_op, layer_ip)
            local_b = np.zeros((layer_op ,1))
            
            self.params.append([local_w, local_b])
            layer_ip = layer_op
        return
    
    def forward_pass(self, X):
        ip = X.copy(deep=True)
        for layer_no in range(self.total_layers):
            
            [W, b] = self.params[layer_no]
            print('Params shapes:',W.shape, b.shape)
            
            H = np.dot(ip, W.T) + b.T
            A = 0
            if layer_no == self.total_layers -1:
                A = self.activation(H, 'sigmoid') #For OP only sigmoid
            else:
                A = self.activation(H, self.activation_m)
            # Do we need to store the layers input and outputs as well?
            
            ip = A
        
        return self.softmax(ip)
    
    def back_propagation(self, Y):
        return None
            
            

In [2]:
class FF_MultiClass_InputWeightVectorised:
  
  def __init__(self, W1, W2):
    self.W1 = W1.copy()
    self.W2 = W2.copy()
    self.B1 = np.zeros((1,2))
    self.B2 = np.zeros((1,4))
  
  def sigmoid(self, X):
    return 1.0/(1.0 + np.exp(-X))
  
  def softmax(self, X):
    exps = np.exp(X)
    return exps / np.sum(exps, axis=1).reshape(-1,1)
  
  def forward_pass(self, X):
    self.A1 = np.matmul(X,self.W1) + self.B1 # (N, 2) * (2, 2) -> (N, 2)
    self.H1 = self.sigmoid(self.A1) # (N, 2)
    self.A2 = np.matmul(self.H1, self.W2) + self.B2 # (N, 2) * (2, 4) -> (N, 4)
    self.H2 = self.softmax(self.A2) # (N, 4)
    return self.H2
    
  def grad_sigmoid(self, X):
    return X*(1-X) 
  
  def grad(self, X, Y):
    self.forward_pass(X)
    m = X.shape[0]
    
    self.dA2 = self.H2 - Y # (N, 4) - (N, 4) -> (N, 4)
    
    self.dW2 = np.matmul(self.H1.T, self.dA2) # (2, N) * (N, 4) -> (2, 4)
    self.dB2 = np.sum(self.dA2, axis=0).reshape(1, -1) # (N, 4) -> (1, 4)
    self.dH1 = np.matmul(self.dA2, self.W2.T) # (N, 4) * (4, 2) -> (N, 2)
    self.dA1 = np.multiply(self.dH1, self.grad_sigmoid(self.H1)) # (N, 2) .* (N, 2) -> (N, 2)
    
    self.dW1 = np.matmul(X.T, self.dA1) # (2, N) * (N, 2) -> (2, 2)
    self.dB1 = np.sum(self.dA1, axis=0).reshape(1, -1) # (N, 2) -> (1, 2)

      
  def fit(self, X, Y, epochs=1, learning_rate=1, display_loss=False):
      
    if display_loss:
      loss = {}
    
    for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      self.grad(X, Y) # X -> (N, 2), Y -> (N, 4)
        
      m = X.shape[0]
      self.W2 -= learning_rate * (self.dW2/m)
      self.B2 -= learning_rate * (self.dB2/m)
      self.W1 -= learning_rate * (self.dW1/m)
      self.B1 -= learning_rate * (self.dB1/m)

      if display_loss:
        Y_pred = self.predict(X)
        loss[i] = log_loss(np.argmax(Y, axis=1), Y_pred)
    
    
    if display_loss:
      plt.plot(loss.values())
      plt.xlabel('Epochs')
      plt.ylabel('Log Loss')
      plt.show()
      
  
  def predict(self, X):
    Y_pred = self.forward_pass(X)
    return np.array(Y_pred).squeeze()